In [0]:
#Imports
from pyspark.sql.types import *
from pyspark.sql.functions import to_timestamp, when, col

#configuration
spark.conf.set(
  "fs.azure.account.key.ecommercestorage5400.dfs.core.windows.net",
  "password"
)

#src path
bronze_path = "abfss://ecommerce@ecommercestorage5400.dfs.core.windows.net/bronze"
#dest path
silver_path = "abfss://ecommerce@ecommercestorage5400.dfs.core.windows.net/silver"

df_bronze = (
    spark.readStream.
    format("delta")
    .load(bronze_path)
)

#cleaned data (Here produced data using faker so not much cleaning required)
df_clean = (
    df_bronze
    .withColumn('timestamp', to_timestamp('timestamp', 'yyyy-MM-dd HH:mm:ss'))
    .withColumn('price',when(col('price').isNull(), 0.0).otherwise(col('price')))
    .withColumn('quantity', when(col('quantity').isNull(), 0).otherwise(col('quantity')))
    .withColumn('total_amount', col('price')*col('quantity'))
    .dropDuplicates(['order_id'])
    .filter(col('country')=='USA')
    .filter(col('state').isNotNull())
)

#Write to silver layer
( df_clean.writeStream
  .format("delta")
  .option("checkpointLocation", f'{silver_path}/_checkpoint')
  .start(silver_path)
)

In [0]:
df_silver=spark.read.format('delta').load(silver_path)
display(df_silver) 

order_id,timestamp,customer_id,product_id,category,price,quantity,total_amount,city,state,country,latitude,longitude,delivery_status
4f4152b0-f09b-40f7-8815-416f61ba6b4d,null,76d2f1a3-375a-4364-bc53-5b4cf25e3c2d,ebba1b54-5dad-4f77-b876-b561a42a53af,Electronics,1321.04,2,2642.08,New York,NY,USA,40.7128,-74.006,Cancelled
1908644b-b1b1-4e0f-9528-9a9f92323a34,null,68fc6642-bc7d-42de-b3ab-f7d6bc6cbc75,5b8ef732-a840-4a29-8758-e18ac57b3f2d,Clothing,469.5,5,2347.5,Los Angeles,CA,USA,34.0522,-118.2437,Processing
b0fb171a-4451-4271-b4a1-8c01db74f8c7,null,aea6ffc0-5e99-4cab-85f7-878501898502,2fc5ce46-dd9e-444b-adfa-f1e9b04b1791,Electronics,1600.88,2,3201.76,Phoenix,AZ,USA,33.4484,-112.074,Cancelled
aa5ed187-7ac3-49dd-b2e9-86ef520e5fb4,null,efde3dc7-16f6-4256-b831-1939eebd5c50,50b2e15e-3b8a-44d2-9b36-295673b2f1e5,Electronics,1957.44,1,1957.44,Los Angeles,CA,USA,34.0522,-118.2437,Cancelled
b1957409-068d-4a7f-a5a9-533f9351a693,null,f74df7c9-ed75-4ad7-9365-32829f45542c,198e0d5a-9cc4-48be-8474-66bf6942b89a,Clothing,154.99,2,309.98,Los Angeles,CA,USA,34.0522,-118.2437,Cancelled
b896a8f2-7b2d-485e-a6b5-cff488327969,null,20d634ab-1ff7-4bfe-b49e-675ff6335c3f,153ec70b-8c3b-4792-a59c-bf379880f794,Electronics,602.68,2,1205.36,Houston,TX,USA,29.7604,-95.3698,Delivered
55daba6c-ec87-4ebe-8e46-9f3f5d44dd5a,null,7605663e-221c-40c8-a571-167cb0ab673b,61df31b1-109f-4559-942c-ce1eab40ddfa,Books,433.1,4,1732.4,Los Angeles,CA,USA,34.0522,-118.2437,Delivered
5acd8147-7cd5-4870-aa3c-e42c6f96efd2,null,05c15cce-72dd-4f17-8ff8-3ec5c97b818e,dbe384ce-3bf6-4c02-a397-c71f11ac44cb,Books,1226.96,4,4907.84,Los Angeles,CA,USA,34.0522,-118.2437,Delivered
876d9dba-9e5f-46df-a322-b1b199ab3a41,null,28977569-96ec-4b37-ad28-d718168b0553,d5434451-0134-42cd-9e98-413ad042b01e,Home Decor,1859.27,2,3718.54,Phoenix,AZ,USA,33.4484,-112.074,Cancelled
dfca16e6-974d-413e-bf92-4f823909862a,null,a855c729-1943-44b7-8cd3-d446eee63c19,d1df1255-7cb3-4476-b3d2-4fd5ceeced66,Clothing,850.28,1,850.28,Houston,TX,USA,29.7604,-95.3698,Cancelled
